In [0]:
from google.colab import drive, files
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
from keras import applications
from keras import optimizers
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, InputLayer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
ROOT_PATH    = 'drive/My Drive/master1/medical_image_recognition/'
DATASET_NAME = 'kvasir_v2'
DATASET_PATH = ROOT_PATH + 'datasets/' + DATASET_NAME + '/'
MODEL_PATH   = ROOT_PATH + 'models/'   + DATASET_NAME + '/'

In [0]:
def get_generator(directory,
                  image_shape=(224, 224),
                  batch_size=32,
                  should_augment=False,
                  data_gen_args=None):
    
    # only rescale
    if should_augment is False:
        image_gen = ImageDataGenerator(rescale=(1./255))
    
    # use dictionary to define augmentations
    else:
        if data_gen_args is None:
            data_gen_args = dict(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
        
        image_gen = ImageDataGenerator(**data_gen_args)
    
    return image_gen.flow_from_directory(directory,
                                         target_size=image_shape,
                                         batch_size=batch_size)

In [0]:
HEIGHT, WIDTH, CHANNELS = 224, 224, 3
BATCH_SIZE = 32

train_generator = get_generator(DATASET_PATH + 'train',
                                batch_size=BATCH_SIZE,
                                should_augment=True)

test_generator = get_generator(DATASET_PATH + 'val',
                               batch_size=BATCH_SIZE)

Found 4800 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


In [0]:
def build_model(image_shape,
                classes,
                num_layers_to_freeze=21,
                base_model_weights_path=None,
                top_model_weights_path=None):
    
    model_name = f'vgg19_{num_layers_to_freeze}'
    
    # build the base model
    if base_model_weights_path is None:
        base_model_weights_path = 'imagenet'
    base_model = applications.vgg19.VGG19(input_shape=image_shape,
                                          weights=base_model_weights_path,
                                          include_top=False)
    print('Load VGG19 as base model')
    
    # load base model weights
    #base_model.load_weights(base_model_weights_path)
    
    # adding classification block on top of base model
    x = Flatten(input_shape=image_shape, name='flatten')(base_model.output)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(216, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    
    # combine both
    model = Model(inputs=base_model.input, outputs=x, name=model_name)
    
    # load top model weights
    #model.load_weights(top_model_weights_path)
    
    # freeze layers
    for i, layer in enumerate(model.layers):
        if i <= num_layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True
            print(f'Layer {i} {layer.name} is trainable')

    # compile model
    loss_type = 'binary_' if classes == 2 else 'categorical_'
    loss_type += 'crossentropy'
    model.compile(loss=loss_type,
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    
    print('Model compiled')
    return model

In [0]:
BASE_MODEL_WEIGHTS_PATH = ROOT_PATH + 'models/vgg19_weights_no_top.h5'

model = build_model(image_shape=(HEIGHT, WIDTH, CHANNELS), classes=8)
model.summary()

In [0]:
# fine-tune the model
EPOCHS = 300
TRAIN_LEN = train_generator.n
TEST_LEN = test_generator.n
CLASS_WEIGHT = None #{0: 1.0, 1: 0.4}

SAVE_HISTORY_PATH = MODEL_PATH + model.name + '.history'
SAVE_MODEL_PATH   = MODEL_PATH + model.name + '.model'

import pickle

def train_model(save_model_path,
                save_history_path,
                save_history=True):
    
    early_stopping = EarlyStopping(patience=20,
                                   monitor='val_loss',
                                   restore_best_weights=True)

    checkpoint = ModelCheckpoint(save_model_path,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=False)

    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
    #                              patience=5, min_lr=0.001)

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=TRAIN_LEN //  BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_generator,
        validation_steps=TEST_LEN // BATCH_SIZE,
        verbose=1,
        callbacks=[early_stopping, checkpoint],
        class_weight=CLASS_WEIGHT)

    # save history
    if save_history:
        with open(save_history_path, 'wb') as file:
            pickle.dump(history.history, file)
        print(f'\n\nSaved history into {save_history_path}')

In [0]:
train_model(save_model_path=SAVE_MODEL_PATH,
            save_history_path=SAVE_HISTORY_PATH)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
150/150 [==============================] - 1454s 10s/step - loss: 2.1941 - acc: 0.1727 - val_loss: 1.8496 - val_acc: 0.3438

Epoch 00001: val_loss improved from inf to 1.84964, saving model to drive/My Drive/master1/medical_image_recognition/models/kvasir_v2/vgg19_21.model
Epoch 2/300
150/150 [==============================] - 188s 1s/step - loss: 1.9385 - acc: 0.2263 - val_loss: 1.7011 - val_acc: 0.3575

Epoch 00002: val_loss improved from 1.84964 to 1.70110, saving model to drive/My Drive/master1/medical_image_recognition/models/kvasir_v2/vgg19_21.model
Epoch 3/300
150/150 [==============================] - 193s 1s/step - loss: 1.8321 - acc: 0.2619 - val_loss: 1.5892 - val_acc: 0.4725

Epoch 00003: val_loss improved from 1.70110 to 1.58924, saving model to drive/My Drive/master1/medical_image_recognition/models/kvasir_v2/vgg19_21.model
Epoch 4/300
150/150 [==============================] - 190s 1s/step - loss: 1.7341 - acc: 